In [218]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
import seaborn as sns
import matplotlib.style as style
from datetime import datetime

pl_march_csv_text = 'data_pl_march_2021.csv' 
amazon_sale_report = 'data_amazon_sale_report.csv' 
may_2022 = 'data_may_2022.csv' 
sale_report = 'data_sale_report.csv' 
international_sale_report = 'data_international_sale_report.csv' 
cloud_warehouse_compersion_chart = 'data_cloud_warehouse_compersion_chart.csv' 
expense_iigf = 'data_expense_iigf.csv' 

def read_csv(name_csv):
    data = pd.read_csv(name_csv,sep=',')
    return data

def extract_size_to_sku(sku,val,pos):
    my_list=sku.split(val)
    if(len(my_list)>0):
        return my_list[pos]
    else:
        return sku

def clean_null_data(dataframe): 
    data_updated = dataframe.replace(['Nill'], np.nan)
    data_cleaned = data_updated.dropna() 
    return data_cleaned

def export_to_csv(data,namefile):
    data.to_csv(namefile,index=False)

Archivo: PL MARCH 2021

Limpieza de valores Nill / Null

Separar el valor de la talla del SKU para tener esto como nuevo Campo

In [219]:

def pl_march_cleaned_data():
    pl_march_data = read_csv(pl_march_csv_text)
    pl_march_clean_data = clean_null_data(pl_march_data) 
    pl_march_clean_data['Size'] = pl_march_clean_data['Sku'].apply(lambda x: extract_size_to_sku(x,'_',-1))
    return pl_march_clean_data

Archivo: Amazon Sale Report

In [220]:
def amazon_sale_report_cleaned_data():
    amazon_sale_report_data = read_csv(amazon_sale_report)
    
    ##Eliminamos las columnas número 17 (índice 16) y 24 (índice 23) debido a su falta de consistencia, completitud y conformidad de la información
    columnas_eliminar = [16 ,23]
    amazon_sale_report_data = amazon_sale_report_data.drop (amazon_sale_report_data.columns [columnas_eliminar] , axis=1)
    
    # Convierte la columna "Date" al formato de fecha de pandas
    amazon_sale_report_data["Date"] = pd.to_datetime(amazon_sale_report_data["Date"])
    return amazon_sale_report_data

Archivo: May_2022

In [221]:
def may_2022_cleaned_data():
    may_2022_data = read_csv(may_2022)
    #Conversion de datos no numericos a numéricos
    columns_to_convert_step_1 = ['MRP Old', 'Ajio MRP', 'Amazon MRP', 'Amazon FBA MRP', 'Flipkart MRP', 'Limeroad MRP', 'Myntra MRP', 'Paytm MRP']

    for column in columns_to_convert_step_1:
        may_2022_data[column] = pd.to_numeric(may_2022_data[column], errors='coerce')
        may_2022_data[column] = may_2022_data[column].astype(float) 

    # completar la conversion de datos no numericos a numéricos
    columns_to_convert_step_2 = ['Weight', 'TP','Final MRP Old','Snapdeal MRP']

    for column in columns_to_convert_step_2:
        may_2022_data[column] = pd.to_numeric(may_2022_data[column], errors='coerce')
        may_2022_data[column] = may_2022_data[column].astype(float)

    """ Limpieza de valores nulos """
    may_2022_clean_data = clean_null_data(may_2022_data) 

    """ Creacion de nueva columna con el ultimo campo de SKU """
    may_2022_clean_data['Size'] = may_2022_clean_data['Sku'].apply(lambda x: extract_size_to_sku(x,'_',-1))
     
    # Obtener el índice de la columna "Style Id"
    style_id_index = may_2022_clean_data.columns.get_loc('Style Id')

    # Verificar si la columna 'Size' está en la lista de columnas
    if 'Size' in may_2022_clean_data.columns:
        # Obtener las columnas en el orden deseado
        new_column_order = may_2022_clean_data.columns.tolist()
        new_column_order.remove('Size')
        new_column_order.insert(style_id_index + 1, 'Size')
        
    # Reordenar las columnas del DataFrame
        may_2022_clean_data = may_2022_clean_data[new_column_order]

    return may_2022_clean_data

Archivo: International Sale Report

In [222]:
def internatonal_sale_report_cleaned_data():
    isr_data = read_csv(international_sale_report)
    print('Data de entrada: ',isr_data.shape)
    ##porcentaje de valores nulos por columna
    round(isr_data.isnull().sum() / isr_data.shape[0] * 100.00,2)

    null_columns=isr_data.columns[isr_data.isnull().any()]
    isr_data[null_columns].isnull().sum()
    isr_data[isr_data.isnull().any(axis=1)][null_columns].head()

    isr_clean_data = clean_null_data(isr_data) 
    ##reviso si se eliminaron todos los nulos
    round(isr_data.isnull().sum() / isr_data.shape[0] * 100.00,2)

    ## en la revisión de datos se puede ver que en la columna SKU, hay un dato con el mimo nombre. Busco la posición de ese dato.
    SKU = isr_clean_data[isr_clean_data['SKU'] == 'SKU' ]
    
    ##mostrar datos desde fila 19675 
    # print(isr_clean_data.loc[isr_clean_data['index']>=int(SKU['index'])])

    # separar en 2 archivos.csv el df, según la posición encontrada.
    k = 2
    size = int(SKU['index'])
    # print('Data inicial ',isr_clean_data.shape)
    # print('Inicio ',k)
    # print('Fin ',size)
    # Generamos los 2 ficheros independientes
    for i in range(k):
        df = isr_clean_data[size*i:size*(i+1)] 
        export_to_csv(df,f'International_Sale_Report{i+1}.csv')
        
    isr_file1 = pd.read_csv("International_Sale_Report1.csv")
    isr_file2 = pd.read_csv("International_Sale_Report2.csv")
    
    isr_file2.columns = ['index', 'CUSTOMER', 'DATE', 'Months', 'Style', 'SKU','PCS','RATE','GROSS AMT','Stock']

    # ## elimina primer registro que tiene el nombre de la columna
    isr_file2.drop([0], axis=0, inplace=True)
    isr_file2['Size'] = isr_file2['SKU'].apply(lambda x: extract_size_to_sku(x,'-',-1))
 
    # ## eliminar columnas 'index','CUSTOMER','DATE','Months','PCS','RATE','GROSS AMT'
    isr_file2 = isr_file2.drop(columns=['index','CUSTOMER','DATE','Months','PCS','RATE','GROSS AMT'])
    # ##Reordenar columnas archivo 2
    isr_file2 = isr_file2.reindex(['SKU','Style','Size','Stock'], axis=1) 
    isr_file2.duplicated()
    isr_file2=isr_file2.drop_duplicates()
    
    ## ULTIPO PASO. Dar formato Fecha a columna DATE
    # isr_file1['DATE'] = pd.to_datetime(isr_file1['DATE'])
    export_to_csv(isr_file1,'International_Sale_Report_1.csv')
    export_to_csv(isr_file2,'International_Sale_Report_2.csv')

    # return [isr_file1,isr_file2]
 

Archivo: Sale Report


In [223]:
def sale_report_cleaned_data():
    sale_report_data = read_csv(sale_report)
    sale_report_data.drop_duplicates(inplace = True)
    sale_report_cleaned_data = clean_null_data(sale_report_data) 
    sale_report_cleaned_data = sale_report_cleaned_data.drop(columns = "index")
    df02 = sale_report_cleaned_data.rename(columns = {'Design No.':'Style'})
    df02 = df02.drop(df02[df02['Size'] == 'FREE'].index)
 
    print(df02.info()) 
    export_to_csv(df02,'Sale_report_cleaned.csv')

In [224]:
def expense_iigf_cleaned_data():
    expense_iigf_data = read_csv(expense_iigf)
    expense_iigf_cleaned_data = expense_iigf_data.drop(columns=['Unnamed: 1','Unnamed: 3'])
    print(expense_iigf_cleaned_data.columns)
    print(expense_iigf_cleaned_data)

In [225]:
def cloud_warehouse_compersion_chart_cleaned_data():
    cloud_warehouse_compersion_chart_data = read_csv(cloud_warehouse_compersion_chart)
    # cloud_warehouse_compersion_chart_cleaned_data = cloud_warehouse_compersion_chart_data.drop(columns=['Unnamed: 1','Unnamed: 3'])
    print(cloud_warehouse_compersion_chart_data.columns)
    print(cloud_warehouse_compersion_chart_data)

In [226]:
""" Ejecucion de cada uno de los reportes """

# print(pl_march_cleaned_data().shape)
# print(amazon_sale_report_cleaned_data().shape)
# print(may_2022_cleaned_data().shape)
# internatonal_sale_report_cleaned_data()
# sale_report_cleaned_data()
# expense_iigf_cleaned_data()
cloud_warehouse_compersion_chart_cleaned_data()

Index(['index', 'Shiprocket', 'Unnamed: 1', 'INCREFF'], dtype='object')
    index                                         Shiprocket  \
0       0                                              Heads   
1       1                      Inbound (Fresh Stock and RTO)   
2       2                                           Outbound   
3       3                                    Storage Fee/Cft   
4       4                   Customer Return with Detailed QC   
5       5  (A) SCOPE OF WORK and STANDARD OPERATING PROCE...   
6       6                                             Inward   
7       7                                                NaN   
8       8                                                NaN   
9       9                                                NaN   
10     10                                                NaN   
11     11                                                NaN   
12     12                                            Outward   
13     13                       